# Extract Transform Load
In this notebook, we extract data from the praise worksheet, transform it into suitable, clean data, and load it into the outputs directory. 

In [36]:
import pandas as pd
import numpy as np
import re

## Total Hours Dataset

In [66]:
total_hours = pd.read_excel("data/TEC Praise Quantification.xlsx", sheet_name="Total Impact Hours so far", engine='openpyxl', header=1, index_col=0, usecols="A:D").dropna(thresh=2).reset_index()
total_hours.to_csv('outputs/total_hours.csv', index=False)
total_hours.head(15)

,Handle,Impact Hours,Payment Status,Estimate IH if nothing was deducted
0,zeptimusq,669.466400,unpaid,NaN
1,santigs67,593.609903,unpaid,NaN
2,cranders71,394.858600,unpaid,NaN
3,ygganderson,371.543022,Partial paid,NaN
4,sembrestels,361.269427,Partial paid,NaN
5,solsista,290.377053,unpaid,NaN
6,iviangita,254.522356,Partial paid,509.044711
7,akrtws,247.108503,unpaid,NaN
8,mateodaza,224.992460,Partial paid,NaN
9,metaverde,204.586181,unpaid,NaN


## Praise Dataset
We consider three batches of praise. Batch one is October 2020, and December2020-May2021. Batch two is prior to December 2020 (other than October). Batch three is after May 7th 2021. Batch one is the original batch that all analysis was performed on. Chronologically, prior to Batch one (batch two) is the early days of praise, and batch 3 is praise that started after the initial praise analysis began.

#### Batch One

In [3]:
periods = [
    "#17 May 7",
    "#16 Apr 24",
    "#15 Apr 9",
    "#14 Mar 26",
    "#13 Mar 12",
    "#12 Feb 26",
    "#11 Feb 12",
    "#10 Jan 29",
    "#9 Jan 15", 
    "#8 Jan 1",
    "#7 Dec 18",
    "#6 Dec 4",
    "#2 Oct 9",

]

In [4]:
def read_period_from_sheet(period: str, batch: str) -> pd.DataFrame:
        # Load the data
        df = pd.read_excel('data/TEC Praise Quantification.xlsx', skiprows=2, sheet_name=period,engine='openpyxl', usecols="A:M")
        
        # Add a period column
        df['period'] = period
        df['batch'] = batch
        
        # Remove the validator normalization as it is confusing and unecessary for analysis
        df.columns = list(df.columns[:6]) + ['v1 norm', 'v2 norm', 'v3 norm'] + list(df.columns[9:])
        df = df.dropna(thresh=8).drop(['v1 norm', 'v2 norm', 'v3 norm'], axis=1)
        
        # Return the loaded df
        return df

In [5]:
# Load and concatenate data
data = pd.concat([read_period_from_sheet(p, batch='Batch 1') for p in periods])

In [6]:
# Combine these duplicate columns and drop the lesser named one.
df = data.copy()
# To
df['To'] = df['To'].combine_first(df['To.1']).combine_first(df['Unnamed: 12'])
df = df.drop(['To.1', 'Unnamed: 12'], axis=1)

# IH Per Praise
df['IH per Praise'] = df['IH per Praise'].combine_first(df['Cred per Praise'])
df = df.drop('Cred per Praise', axis=1)

# IH Per Person Per Period
df['IH per person'] = df['IH per person'].combine_first(df['Cred per person'])
df = df.drop('Cred per person', axis=1)

# Rename The Institution Column
df = df.rename({'Unnamed: 3':'Institution'}, axis=1)

#### Batch Two

In [8]:
df.head(1)

,To,From,Reason for dishing,Institution,Date,Room,Avg %,IH per Praise,IH per person,period,batch
0,VitorMarthendal,iviangita#3204,for pushing to get the Hatch Dashboard out,Token Engineering Commons,May-07-2021,🙏praise,0.022117,26.540405,7.559265,#17 May 7,Batch 1


In [9]:
periods = [
    "#5 Nov 20", 
    "#4 Nov 6", 
    "#3 Oct 23", 
    "#1 Sept 24", 
    "#0 Sept 7 (historic)", 
]

In [10]:
data2 = pd.concat([read_period_from_sheet(p, batch='Batch 2') for p in periods])

In [11]:
# Combine these duplicate columns and drop the lesser named one.
df2 = data2.copy()
# To
df2['To'] = df2['To'].combine_first(df2['To.1'])
df2 = df2.drop(['To.1'], axis=1)

# IH Per Praise
df2 = df2.rename({'Cred per Praise':'IH per Praise'}, axis=1)

# IH Per Person Per Period
df2 = df2.rename({'Cred per person':'IH per person'}, axis=1)

# Rename The Institution Column
df2 = df2.rename({'Unnamed: 3':'Institution'}, axis=1)

#### Batch 3

In [13]:
periods = [
    "#18 May 21",
    "#19 Jun 4",
    "#20 Jun 18",
]

In [14]:
data3 = pd.concat([read_period_from_sheet(p, batch='Batch 3') for p in periods])

In [16]:
# Combine these duplicate columns and drop the lesser named one.
df3 = data3.copy()
# To
df3['To'] = df3['To'].combine_first(df3['Unnamed: 12'])
df3 = df3.drop(['Unnamed: 12'], axis=1)

# Rename The Institution Column
df3 = df3.rename({'Unnamed: 3':'Institution'}, axis=1)

### Concatenate Praise Batches into Praise Dataset

In [29]:
data = pd.concat([df, df2, df3])

### Resolving Names

In [50]:
names_df = pd.read_excel('data/TEC Praise Quantification.xlsx',sheet_name="DO NOT TOUCH Imported",engine='openpyxl', usecols="A:D")

#### We create a dictionary that matches each non-null entry in the spreadsheet to its "IH & CSTK Handle"
source_cols = names_df.columns
names_dict = {} #create a blank dictionary

for i in range(len(names_df)):
    for col in source_cols:
        name_to_consider = names_df.loc[i, col]
        canonical_name = names_df.loc[i,"IH & CSTK Handle"]
        if pd.isna(canonical_name):
            canonical_name = name_to_consider
        if not(pd.isna(name_to_consider)):
            names_dict[name_to_consider] = canonical_name

#### Did we catch them all? See if there's anything in the combned user sets of "To" and "From" that isn't a key in the names_dict dictionary. 
combined_users = set([]).union(set(receivers["From"]),set(receivers["To"]))
names_uncaught = sorted([str(x) for x in combined_users if (not(x in names_dict.keys()) and not(pd.isna(x)))])

# print("All told, there are {} names in our data set that lack canonical representations: \n".format(str(len(names_uncaught))))
# print("We do not have canonical representations for the following names: \n")
# for name in names_uncaught:
#     print(str(name))

def clean_name(name):
    new_name = str(name)
    new_name = new_name.lower()
    new_name = re.sub('#\d\d\d\d','',new_name)
    new_name = re.sub('[^A-Za-z0-9]+', '', new_name) #remove all non-alphanumeric characters
    return new_name

clean_name("zeptimusQ")
clean_name("ygg_anderson")
clean_name("AmwFund#0979")
clean_name("Caeser (PST)#0046")

cleaned_keys = []
original_keys = list(names_dict.keys())
for key in original_keys:
    clean_key = clean_name(key)
    cleaned_keys.append(clean_key)
    if not(clean_key == key):
        names_dict[clean_key] = names_dict[key]
    
new_pairs = {}

for name in names_uncaught:
    clean_version = clean_name(name) #clean the name 
    key_matches = [key for key in cleaned_keys if clean_version == key] #make a list of all keys with same cleaned name
    if len(key_matches) > 0:
        for match in key_matches:
            if match in names_dict.keys():
                right_name = names_dict[match]
                new_pairs[name] = right_name
                names_dict[name] = right_name
                break
                    
# print("We were able to add the following names: \n")
# for name in new_pairs.keys():
#     print(str(name) + "\t ----> \t" + str(names_dict[name]))

still_uncaught_names = sorted([str(x) for x in combined_users if (not(x in names_dict.keys()))])
num_still_uncaught_names = str(len(still_uncaught_names))
print("We are still missing {} names".format(num_still_uncaught_names))

We are still missing 11 names


In [51]:
def total_appearances(name, data):
    from_appearances = sum(data["From"] == name)
    to_appearances = sum(data["To"] == name)
    total = from_appearances + to_appearances
    return total 

still_uncaught_appearances = [total_appearances(x, data) for x in still_uncaught_names]

uncaught_df = pd.DataFrame({"name": still_uncaught_names, "appearances" : still_uncaught_appearances})

for name in still_uncaught_names:
    names_dict[name] = name

last_remaining = [user for user in combined_users if(not(user in names_dict.keys()))]
num_remaining = len(last_remaining)
print("There are {} remaining users with no representation.".format(num_remaining))
print("These users are \n")

for name in last_remaining:
    print(name)


There are 0 remaining users with no representation.
These users are 



In [85]:
data.loc[:,"To"] = data.loc[:,"To"].map(names_dict)
data.loc[:, "From"] = data.loc[:,"From"].map(names_dict)

### Save Cleaned Praise Dataset

In [86]:
data.to_csv('outputs/praise_data.csv',index=False)

### Split Receivers and Quantifiers

In [87]:
receivers = data[(~data[['Institution', 'Date', 'Room']].isna().all(axis=1)) & (data['From'] != 'Quantifiers')]
quantifiers = data[(data[['Institution', 'Date', 'Room']].isna().all(axis=1)) | (data['From'] == 'Quantifiers')]
len(receivers) + len(quantifiers) == len(data)

True

In [90]:
receivers.to_csv('outputs/receivers.csv', index=False)
quantifiers.to_csv('outputs/quantifiers.csv', index=False)